In [ ]:
#importar
import os
from datetime import datetime
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import os
from bs4 import BeautifulSoup
import re
from collections import OrderedDict
from selenium.common.exceptions import NoSuchElementException
import getpass
import pyshorteners

: 

In [ ]:
def realizar_login(url, login1, password1, orgao1):
    """
    Função para realizar login no sistema SEI.

    Args:
        url (str): URL do sistema SEI.
        login1 (str): Nome de usuário para login.
        password1 (str): Senha para login.
        orgao1 (str): Nome do órgão para acesso.

    Returns:
        webdriver: Instância do WebDriver com o usuário autenticado.
    """
    try:
        # Inicializa o navegador
        driver = webdriver.Chrome()
        driver.implicitly_wait(0.5)
        driver.get(url)

        # Localiza os elementos de login
        login = driver.find_element(By.XPATH, '//*[@id="txtUsuario"]')
        password = driver.find_element(By.XPATH, '//*[@id="pwdSenha"]')
        orgao = driver.find_element(By.XPATH, '//*[@id="selOrgao"]')
        submit_button = driver.find_element(By.XPATH, '//*[@id="Acessar"]')

        # Preenche as credenciais
        login.send_keys(login1)
        password.send_keys(password1)
        orgao.send_keys(orgao1)

        # Realiza o login
        submit_button.click()
        time.sleep(3)  # Aguarda carregamento da página após o login

        print("Login realizado com sucesso!")
        return driver
    except Exception as e:
        print(f"Erro ao realizar o login: {e}")
        return None

# Função principal para busca
def buscar_arquivos(driver):
    """
    Realiza a busca de arquivos no sistema SEI após login.

    Args:
        driver (webdriver): Instância do WebDriver autenticada.
    """
    try:
        # Acessa a área de busca
        searching = driver.find_element(By.XPATH, '//*[@id="infraMenu"]/li[14]/a/span')
        searching.click()
        time.sleep(1)

        # Restringe busca ao órgão específico
        sel_orgao = driver.find_element(By.XPATH, '//*[@id="divSinRestringirOrgao"]/div')
        sel_orgao.click()
        time.sleep(1)

        # Especifica os termos de pesquisa
        espec_pesq = driver.find_element(By.XPATH, '//*[@id="txtDescricaoPesquisa"]')
        espec_pesq.send_keys('MPS e SEGES não "Serviço de Informações ao Cidadão" não "Capacitação" não "Avaliação de Reação" não "Termo de Responsabilidade - Controle de Acesso" não "Solicitação de Cessão" não "TERMO ANUÊNCIA"')
        
        #colocar como tramitação dentro do orgão
        chktram = driver.find_element(By.XPATH, '//*[@id="divSinTramitacao"]/div')
        chktram.click()
        time.sleep(0.2)

        # Realiza a pesquisa
        b_pesq = driver.find_element(By.XPATH, '//*[@id="sbmPesquisar"]')
        b_pesq.click()
        time.sleep(3)

        print("Busca realizada com sucesso.\nRestringindo em PL e dentro do MGI.\n\nOs Externo entram como MGI.")
    except Exception as e:
        print(f"Erro durante a busca: {e}")

# Exemplo de uso
url = 'hhttps://colaboragov.sei.gov.br/sip/modulos/MF/login_especial/login_especial.php?sigla_orgao_sistema=MGI&sigla_sistema=SEI'
login1 = ''
password1 = ""
orgao1 = "MGI"

# Realiza login
driver = realizar_login(url, login1, password1, orgao1)

# Se o login foi bem-sucedido, realiza a busca
if driver:
    buscar_arquivos(driver)

: 

In [3]:
def extrair_dados(driver):     
    """
    Função para realizar web scraping no SEI e retornar os dados em um DataFrame.

    Args:
        driver (webdriver): Instância do Selenium WebDriver.

    Returns:
        pd.DataFrame: DataFrame contendo os dados extraídos.
    """
    def remove_items(lista, item): 
        """Remove todos os itens iguais a `item` de uma lista."""
        return [i for i in lista if i != item]

    # Extraindo os elementos da pesquisa
    tree_elements = driver.find_elements("xpath", '//*[@class="pesquisaTituloEsquerda"]/a')
    list_tree = [element.text for element in tree_elements]
    trees = remove_items(list_tree, '')  # Remover elementos vazios

    abts = driver.find_elements("xpath", '//*[@class="pesquisaSnippet"]')
    list_abts = [element.text for element in abts]

    unidades = driver.find_elements("xpath", '//*[@class="pesquisaMetatag"]')
    list_uni = [element.text.split(':') for element in unidades]
    info = [sublist_uni[1] for sublist_uni in list_uni if len(sublist_uni) > 1]  # Removendo listas vazias

    rows = driver.find_elements("xpath", '//*[@id="conteudo"]/table/tbody/tr')
    links = []
    for i in range(1, len(rows), 3):
        try:
            a = driver.find_element("xpath", f'//*[@id="conteudo"]/table/tbody/tr[{i}]/td[1]/a[1]')
            time.sleep(0.5)
            link = a.get_attribute('href')
            time.sleep(0.5)
            links.append(link)
        except Exception as e:
            print(f"Erro ao processar a linha {i}: {e}")

    shortener = pyshorteners.Shortener()
    links_curtos = []
    for link in links:
        try:
            link_curto = shortener.tinyurl.short(link)
            links_curtos.append(link_curto)
            time.sleep(3)
        except Exception as e:
            print(f"Erro ao encurtar o link {link}: {e}")

    dados = {
        "Número do Processo": trees[::2],
        "Documento": trees[1::2],
        "Resumo": list_abts,
        "Unidade": info[::3],
        "Usuário": info[1::3],
        "Data de Inclusão": info[2::3],
        "Links": links_curtos
    }

    df = pd.DataFrame(dados)
    return df

In [4]:
import os
import time
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# --- Configuração de Logging ---
# Usar logging é melhor do que 'print' para rastrear o que o script está fazendo,
# especialmente para erros.
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("automacao_sei.log"),
        logging.StreamHandler()
    ]
)

In [5]:
def criar_pastas(pasta_documentos, pasta_listas):
    """Garante que os diretórios para salvar os arquivos existam."""
    os.makedirs(pasta_documentos, exist_ok=True)
    os.makedirs(pasta_listas, exist_ok=True)
    logging.info(f"Pasta de documentos '{pasta_documentos}' e de listas '{pasta_listas}' prontas.")


def salvar_documentos_da_pagina(driver, pasta_documentos, caminho_lista_arquivos):
    """
    Localiza todos os documentos na página de resultados de busca atual do SEI,
    salva cada um como um arquivo HTML e registra os nomes em um arquivo de texto.

    Esta função substitui a sua função 'get_files' original.

    Args:
        driver (webdriver): A instância do WebDriver do Selenium.
        pasta_documentos (str): O caminho da pasta onde os arquivos HTML serão salvos.
        caminho_lista_arquivos (str): O caminho completo do arquivo .txt para registrar os nomes dos arquivos.
    """
    janela_original = driver.current_window_handle
    
    try:
        # Em vez de pegar todo o texto, encontramos cada 'tabela' que representa um resultado.
        # Isto é mais robusto que usar um índice no XPath.
        tabelas_de_resultado = driver.find_elements(By.XPATH, "//div[@id='conteudo']/table")
        
        if not tabelas_de_resultado:
            logging.info("Nenhum documento encontrado na página atual.")
            return

        logging.info(f"Encontrados {len(tabelas_de_resultado)} documentos na página.")

        for tabela in tabelas_de_resultado:
            nome_doc = "nome_nao_encontrado"
            try:
                # 1. Encontrar o nome do documento e o link para o processo
                # O seletor busca pelo link que tem o tooltip, que geralmente contém o nome.
                elemento_nome = tabela.find_element(By.XPATH, ".//a[contains(@onmouseover, 'return infraTooltipMostrar')]")
                nome_doc = elemento_nome.text.strip()

                # Limpa o nome do documento para criar um nome de arquivo válido
                nome_arquivo_seguro = "".join([c for c in nome_doc if c.isalnum() or c in (' ', '-')]).rstrip()
                
                # 2. Encontrar o link para abrir a página do documento
                # O XPath original usava a[2], então mantemos uma lógica parecida, mas relativa à tabela.
                elemento_link_doc = tabela.find_element(By.XPATH, ".//tr[1]/td[1]/a[2]")
                link_doc = elemento_link_doc.get_attribute('href')

                if not link_doc:
                    logging.warning(f"Não foi possível encontrar o link para o documento: {nome_doc}")
                    continue

                # 3. Salvar o nome do arquivo na lista de texto
                with open(caminho_lista_arquivos, 'a', encoding='utf-8') as f:
                    f.write(nome_arquivo_seguro + '\n')

                # 4. Abrir o documento em uma nova aba, salvar o HTML e fechar
                driver.switch_to.new_window('tab')
                driver.get(link_doc)
                
                # Use UTF-8, é um padrão mais moderno e evita problemas com caracteres especiais.
                caminho_html = os.path.join(pasta_documentos, f"documento_{nome_arquivo_seguro}.html")
                with open(caminho_html, 'w', encoding='utf-8') as f:
                    f.write(driver.page_source)
                
                logging.info(f"Documento salvo: {caminho_html}")
                
                driver.close()
                driver.switch_to.window(janela_original)
                time.sleep(1) # Pequena pausa para estabilidade

            except NoSuchElementException:
                logging.error(f"Não foi possível encontrar o nome ou o link para um documento na tabela. Pulando.")
                # Garante que, se uma nova aba foi aberta, ela seja fechada antes de continuar.
                if len(driver.window_handles) > 1:
                    driver.close()
                    driver.switch_to.window(janela_original)
                continue
            except Exception as e:
                logging.error(f"Ocorreu um erro ao processar o documento '{nome_doc}': {e}")
                if len(driver.window_handles) > 1:
                    driver.close()
                    driver.switch_to.window(janela_original)
                continue

    except Exception as e:
        logging.error(f"Ocorreu um erro geral ao tentar salvar os documentos da página: {e}")

def navegar_e_salvar_documentos(driver, pasta_documentos, pasta_listas):
    """
    Navega pelas páginas de resultados, chamando a função para salvar 
    os documentos de cada página.

    Esta função substitui a sua 'navegar_paginas'.
    """
    from datetime import datetime
    
    today_date = datetime.now().strftime('%Y-%m-%d')
    caminho_lista_arquivos = os.path.join(pasta_listas, f'lista_arquivos_{today_date}.txt')

    pagina_atual = 1
    while True:
        logging.info(f"--- Processando página {pagina_atual} ---")
        
        # Chama a função para salvar os documentos da página atual
        salvar_documentos_da_pagina(driver, pasta_documentos, caminho_lista_arquivos)

        try:
            # Procura o botão "Próxima"
            botao_proxima = driver.find_element(By.XPATH, "//a[text()='Próxima']")
            
            # Clica no botão para ir para a próxima página
            logging.info("Indo para a próxima página...")
            botao_proxima.click()
            time.sleep(5)  # Aguarda o carregamento da próxima página
            pagina_atual += 1

        except NoSuchElementException:
            logging.info("Botão 'Próxima' não encontrado. Fim da navegação.")
            break  # Sai do loop se o botão "Próxima" não existir
        except Exception as e:
            logging.error(f"Erro ao tentar navegar para a próxima página: {e}")
            break

In [ ]:
def extract_words_after_lei(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text()
        words = text.split()
        if "Lei" in words:
            lei_index = words.index("Lei")
            return ' '.join(words[lei_index + 1:lei_index + 3])
        else:
            return None
    except requests.RequestException as e:
        print(f"Error fetching URL {url}: {e}")
        return None

In [6]:
def get_files():
    
        original_window = driver.current_window_handle 
        page_docs_search = driver.find_element("xpath", '//*[@id="conteudo"]')
        page_docs = page_docs_search.text.split('Atividade Fim: ')[1:]
        
        file_name = []
        for i in range(len(page_docs)):
            a = page_docs[i].split('(REMA - Empréstimo de Materiais ou Ex. Geológicos) ')[1].split('\n1')[0]
            file_name.append(a)
        with open(os.path.join(".", nomes_arquivos, f'{today_date}.txt'), 'a') as file:
            file.write(a +'\n')
        # file.close()

        links = []
        for i in range(1, len(page_docs)+1):
            a = driver.find_element("xpath", '//*[@id="conteudo"]/table[%s]/tbody/tr[1]/td[1]/a[2]' %i)
            link = a.get_attribute('href')
            links.append(link)
        
        index = -1
        for i in links:
            index +=1
            name = file_name[index]
            driver.switch_to.new_window('tab')
            driver.get(i)
            with open(os.path.join(".", pasta, f"documento_{name}.html"), 'w', encoding='iso-8859-1') as file:
                file.write(driver.page_source)        
            driver.close()
            driver.switch_to.window(original_window)
    


In [7]:
def navegar_paginas(driver):
    """
    Loop para navegar por todas as páginas até que não haja mais um botão 'Próxima'.
    Retorna um DataFrame consolidado com os dados de todas as páginas.
    """
    dados_consolidados = pd.DataFrame()  # DataFrame vazio para acumular os dados

    while True:
        try:
            # Extrair os dados da página atual e adicionar ao DataFrame consolidado
            df_pagina = extrair_dados(driver)
            dados_consolidados = pd.concat([dados_consolidados, df_pagina], ignore_index=True)

            # Procurar o botão "Próxima"
            next_page = driver.find_element("xpath", '//*[@id="conteudo"]/div[2]/div[3]/a')

            # Verificar se o botão "Próxima" tem o atributo 'href'
            proxima_href = next_page.get_attribute('href')
            time.sleep(3)
            if not proxima_href:
                print("Não há mais páginas. Encerrando navegação.")
                break  # Sai do loop se não houver link para a próxima página

            # Clicar no botão "Próxima"
            next_page.click()
            time.sleep(10)  # Aguarda o carregamento da próxima página

        except NoSuchElementException:
            print("Botão 'Próxima' não encontrado. Encerrando navegação.")
            break  # Sai do loop se o botão "Próxima" não existir

        except Exception as e:
            print(f"Erro inesperado: {e}")
            break  # Sai do loop em caso de erro inesperado

    # Fechar o navegador após o término
    # driver.close()
    # driver.quit()

    return dados_consolidados

In [ ]:
dados = navegar_paginas(driver)

In [ ]:
dados